# Cálculo de Zapata en Suelo

En este numeral se presentan las bases para el análisis geotécnico de las cimentaciones de las torres de las Líneas de Transmisión 

## SELECCIÓN PROPIEDADES DE DISEÑO

Para el diseño de la cimentación de torres de líneas de transmisión, es necesario determinar o estimar ciertas propiedades geotécnicas de los materiales de fundación. Los parámetros requeridos para calcular la capacidad portante, la capacidad a la tensión (o arrancamiento), la capacidad de carga lateral, asentamientos y todas aquellas variables que puedan influir en la estabilidad de la cimentación, de acuerdo con su aplicabilidad en cada caso, se pueden obtener a partir de correlaciones con varias propiedades índice del suelo en el que se construye la cimentación, apoyados con ensayos de laboratorio de clasificación, resistencia, deformabilidad y compactación.

In [1]:
import ipywidgets as widgets
widgets.IntSlider()

IntSlider(value=0)

### Parámetros de resistencia efectivos del suelo ($N_{SPT}$ de diseño)

Los valores de $N$ de diseño se definen teniendo en cuenta los resultados del ensayo SPT entre los $3.0$ y  $6.0 m$ de profundidad, para cimentaciones superficiales, y entre los $3.0$ y $15.0 m$ de profundidad para el caso de cimentaciones profundas. Para todos los tipos de cimentación, se consideran los valores representativos de $N_{SPT}$, a partir de $3.0 m$ de profundidad, ya que corresponde al nivel de cimentación promedio de las cimentaciones, es decir, se descartan valores que no se encuentren dentro de un rango homogéneo y que sean característicos de los materiales encontrados en la exploración del subsuelo.


Los factores de corrección que se aplican para el cálculo del $N$ utilizado para los diseños de los diferentes tipos de cimentación considerados para el proyecto, se definen de acuerdo con el tipo de material presente, suelos granulares o cohesivos.


$$ N_{crr} = N1 = N * C_n * \eta_1 * \eta_2 * \eta_3 * \eta_4 \qquad(Bowles, 1988)$$

Donde, 

$N_{crr}$ = valor de $N$ corregido

$N$ = valor de $N$ de campo

$\eta_1$ = factor por energía del martillo $(0.45 \leq \eta_1 \leq 1)$

$\eta_2$ = factor por longitud de la varilla $(0.75 \leq \eta_2 \leq 1)$

$\eta_3$ = factor por revestimiento interno de tomamuestras $(0.8 \leq \eta3 \leq 1)$

$\eta_4$ = factor por diámetro de la perforación $(\eta_4 > 1$ para $D> 5”, \eta_4 = 1.15$ para $D=8")$

$C_n$ = factor de corrección por confinamiento efectivo. Se recomienda que sea menor o igual de $2.0$.



$$C_n = \left(\frac{1}{R_s}\right)^{0.5} \qquad (Liao-Whitman)$$


$$ R_s = \frac{\sigma v'}{pa}$$

Donde:

$\sigma v'$ = Esfuerzo vertical efectivo.

$pa$ = Presión atmosférica


# CIMENTACIÓN CON ZAPATAS CONVENCIONALES EN SUELO

La cimentación mediante zapata convencional consiste en una zapata en concreto reforzado, apoyada sobre depósitos de suelo a una profundidad entre dos y cinco metros $(2.0 - 5.0 m)$, por debajo del nivel de terreno natural. Para esta cimentación se determina para cada sitio la capacidad portante admisible, la capacidad por tensión o arrancamiento, asentamientos y se realiza la verificación al volcamiento.

Los análisis de capacidad portante, arrancamiento y carga lateral serán realizados teniendo en cuenta las metodologías presentadas en el IEEE _Guide for transmission structure foundation design and testing_ en el Capítulo 4 Design of spread foundations.

## Capacidad portante

La capacidad portante última en condiciones drenadas está dada por la siguiente expresión (IEEE, 2001):

$$
q_{ult} = \frac{Q}{B'L'} = \frac{1}{2} \gamma BN_{\gamma} \xi_{\gamma s} \xi_{\gamma d} \xi_{\gamma r}
\xi_{\gamma i} \xi_{\gamma t} \xi_{\gamma g} + 
qN_{q} \xi_{qs} \xi_{qd} \xi_{qr} \xi_{qi} \xi_{qt} \xi_{qg}
$$
Donde, 

$q_{ult}$ = Capacidad portante última.

$Q $= Carga máxima (incluye la carga de la estructura, peso efectivo de la fundación y peso del relleno con su volumen).

$B$ = Ancho de la zapata.

$L $= largo de la zapata.

$D $= profundidad de la cimentación.

$B' y L'$ = Son las dimensiones reducidas de B y L debido a la excentricidad de la cimentación.

$\gamma$ = Peso efectivo promedio del suelo para la profundidad de cimentación D.

$q$ = Esfuerzo de sobrecarga efectivo a la profundidad de cimentación D.

$N_{\gamma}$ y $N_{q}$ = Factores de capacidad portante.

$\xi_{xy}$ = Factores de corrección por forma, profundidad, rigidez, inclinación de la carga, inclinación de la base de la cimentación y por la pendiente de terreno.

Así mismo, la capacidad portante última en condiciones no drenadas está dada por la siguiente expresión (IEEE.2001):

In [2]:
import math

def zapata_suelo(Qreq, Trequ, Freq, TipoMat, Sumergencia, IP, N, Eta_2, Eta_3, Eta_4, Theta, Gm_fund, Gm_rell,\
                 Gm_conc, alfa, omega, Df, B, Lpe, AltPedestal, e, FS_cp, FS_arr, FS_cat):
    """
    Calcula las cargas admisibles de una cimentación tipo Zapata en suelo.

    Parameters:
        Qreq: Solicitación de carga axial a compresión [Ton-f]
        Trequ: Solicitación de carga axial a tensión [Ton-f]
        Freq: Solicitación de carga lateral [Ton-f]
        TipoMat: Tipo de material. Puede ser "Granular" o "Cohesivo"
        Sumergencia: Condición de saturación. Puede ser "Sumergido" o "No sumergido"
        IP: Indice de Plasticidad
        N: Número de Golpes (Campo)
        Eta_2: Factor por longitud de la varilla (0.75 ≤ Eta_2  ≤ 1)
        Eta_3: Factor por revestimiento interno de tomamuestras (0.8 ≤ Eta_3 ≤ 1)
        Eta_4: Factor por diámetro de la perforación ( Eta_4 > 1 para Diam. > 5”, y  Eta_4 = 1.15 para Diam. = 8")
        Theta: Ángulo de Inclinación de la Carga con la vert. [Grados]
        Gm_fund: Peso unitario del suelo de fundación [Ton/m³]
        Gm_rell: Peso unitario del suelo de relleno [Ton/m³]
        Gm_conc: Peso unitario del concreto reforzado [Ton/m³]
        alfa: Ángulo de Inclinación de la base. [Grados]
        omega: Ángulo de Inclinación del Terreno. [Grados]
        Df: Profundidad de cimentación [m]
        B: Ancho de cimentación [m]
        Lpe: Lado del Pedestal [m]
        AltPedestal: Altura del pedestal [m]
        e: Espesor de la zapata [m]
        FS_cp: Factor de seguridad a la capacidad portante
        FS_arr: Factor de seguridad al arrancamiento
        FS_cat: Factor de seguridad a la carga axial a tensión

    Returns:
        Diccionario con las siguientes entradas
            N60: Número de Golpes (Corregido)
            f1: Factor de Stround para cálculo de Cu
            Cu: Cohesión (Stround , 1974) [Ton/m²]
            Gm_fund_ec: Peso unitario efectivo del suelo de fundación para compresión [Ton/m³]
            Gm_rell_ec: Peso unitario efectivo del suelo de relleno para compresión [Ton/m³]
            Gm_conc_e: Peso unitario efectivo del concreto reforzado [Ton/m³]
            Phi: Ángulo de fricción (Japan) [Grados]
            Nq: Factor de capacidad por sobrecarga
            Nc: Factor de capacidad por cohesión
            Ng: Factor de capacidad por peso propio
            F_sc: Factor de Corrección por Forma-C
            F_sq: Factor de Corrección por Forma-q
            F_sg: Factor de Corrección por Forma-γ
            F_dq: Factor de Corrección por Profundidad-q
            F_dc: Factor de Corrección por Profundidad-C
            F_dg: Factor de Corrección por Profundidad-γ
            F_iq: Factor de Corrección por Inclinación-q
            F_ic: Factor de Corrección por Inclinación-c
            F_ig: Factor de Corrección por Inclinación-γ
            F_tg: Factor de Corrección por Inclinacion en la base-γ
            F_tq: Factor de Corrección por Inclinacion en la base-q
            F_tc: Factor de Corrección por Inclinacion en la base-c
            F_gq: Factor de Corrección por Inclinacion Terreno-q
            F_gc: Factor de Corrección por Inclinacion Terreno-c
            F_gg: Factor de Corrección por Inclinacion Terreno-γ
            Psi: Ángulo de fricción del cono [Grados]
            Gm_fund_ea: Peso unitario efectivo del suelo de fundación para arrancamiento [Ton/m³]
            Gm_rell_ea: Peso unitario efectivo del suelo de relleno para arrancamienro [Ton/m³]
            Ka: Carga lateral admisible
            Kp: Carga lateral admisible
            Zc: Grieta de Tracción
            C1: Factor de corrección
            v: Relación de Poisson
            E: Módulo de Elasticidad (Stround, 1989; Kulhawy 1990) [Ton-f/m²]
            F_rq: Factor de Corrección por Rigidez-q
            F_rc: Factor de Corrección por Rigidez-c
            F_rg: Factor de Corrección por Rigidez-γ
            Qadm: Capacidad axial a compresión admisible [Ton-f/m²] - qult
            Mest: Momentos de Estabilizacion  [Ton-m]
            Si: Asentamiento (IEEE Std 691-2001) [mm]
            Mvolc: Momentos de Vuelco [Ton-m]
            Faxc_adm: Capacidad axial a compresión admisible [Ton-f]
            Faxt_adm: Capacidad axial a tensión admisible [Ton-f]
            Flat_adm: Carga lateral admisible [Ton-f]
            Sp: Asentamiento Permitido [mm]
            Mest_min: Momento estabilizador Minimo [Ton-m]
    """
    # Theta en radianes
    Theta_rad = math.radians(Theta)

    # alfa en radianes
    alfa_rad = math.radians(alfa)

    # omega en radianes
    omega_rad = math.radians(omega)

    # Gm_agua: Peso unitario del agua [Ton/m³]
    Gm_agua = 1.0

    # N60: Número de Golpes (Corregido)
    N60 = N * Eta_2 * Eta_3 * Eta_4 * 45 / 60

    # f1: Factor de Stround para cálculo de Cu
    if IP > 30:
        f1 = 0.043 * 98.5
    else:
        f1 = 0.00985 * IP**2 - 0.59297 * IP + 13.36645

    # Cu: Cohesión (Stround , 1974) [Ton/m²]
    if TipoMat == "Cohesivo":
        Cu = N60 * f1 / 9.81
    else:
        Cu = 0
        
    # Gm_fund_ec: Peso unitario efectivo del suelo de fundación para compresión [Ton/m³]
    if Sumergencia == "Sumergido":
        Gm_fund_ec = Gm_fund - Gm_agua
    else:
        Gm_fund_ec = Gm_fund

    # Gm_rell_ec: Peso unitario efectivo del suelo de relleno para compresión [Ton/m³]
    if Sumergencia == "Sumergido":
        Gm_rell_ec = Gm_rell - Gm_agua
    else:
        Gm_rell_ec = Gm_rell

    # Gm_conc_e: Peso unitario efectivo del concreto reforzado [Ton/m³]
    if Sumergencia == "Sumergido":
        Gm_conc_e =  Gm_conc - Gm_agua
    else:
        Gm_conc_e =  Gm_conc

    # Phi: Ángulo de fricción (Japan) [Grados]
    if TipoMat == "Cohesivo":
        Phi = 0
    else:
        Phi = math.ceil(math.sqrt(9.375 * ((N60 * 60 / 45) * min(2, math.sqrt(98.5 / (Gm_fund_ec * Df * 10))))) + 15)
    Phi_rad = math.radians(Phi)

    # Nq: Factor de capacidad por sobrecarga
    Nq = math.tan(math.radians(45 + 0.5 * Phi))**2 * math.exp(math.pi * math.tan(Phi_rad))

    # Nc: Factor de capacidad por cohesión
    Nc = (2 + math.pi) if Phi == 0 else ((Nq - 1) * ( 1 / math.tan(Phi_rad)))

    # Ng: Factor de capacidad por peso propio
    Ng = 2 * (Nq + 1) * math.tan(Phi_rad)

    # F_sc: Factor de Corrección por Forma-C
    F_sc = 1 + (Nq / Nc)

    # F_sq: Factor de Corrección por Forma-q
    F_sq = 1 + math.tan(Phi_rad)

    # F_sg: Factor de Corrección por Forma-γ
    F_sg = 1 - 0.4

    # F_dq: Factor de Corrección por Profundidad-q
    F_dq = 1 + 2 * math.tan(Phi_rad) * (1 - math.sin(Phi_rad))**2 * (Df/B if Df/B <= 1 else math.atan(Df/B))                  

    # F_dc: Factor de Corrección por Profundidad-C
    if Phi == 0:
        F_dc = 1 + 0.33 * (math.atan(Df / B))
    else:
        F_dc = F_dq - (1 - F_dq) / (Nc * math.tan(Phi_rad))

    # F_dg: Factor de Corrección por Profundidad-γ
    F_dg = 1

    # F_iq: Factor de Corrección por Inclinación-q
    if Phi == 0:
        F_iq = (1 - ((math.sin(Theta_rad) * Qreq) / (math.cos(Theta_rad)*Qreq)))**((3 / 2) * (0.5 + (0.5 * math.cos(2 * Theta_rad))) + ((3 / 2) * 2 * math.sin(Theta_rad) * math.cos(Theta_rad)))
    else:
        F_iq = (1 - ((math.sin(Theta_rad) * Qreq) / ((math.cos(Theta_rad)*Qreq) + (B**2 * (1 / math.tan(Phi_rad)))))**((3 / 2) * (0.5 + (0.5 * math.cos(2 * Theta_rad))) + ((3 / 2) * 2 * math.sin(Theta_rad))))

    # F_ic: Factor de Corrección por Inclinación-c
    if Phi == 0:
        F_ic = 1-((((3 / 2) * (0.5 + (0.5 * math.cos(2 * Theta_rad)))+((3 / 2) * 2 * math.sin(Theta_rad) * math.cos(Theta_rad))) * (math.sin(Theta_rad) * Qreq))/(5.14 * Cu * B**2))
    else:
        F_ic = F_iq - (1 - F_iq) / (Nc * math.tan(Phi_rad))
                    
    # F_ig: Factor de Corrección por Inclinación-γ
    if TipoMat == "Granular":
        F_ig = 1 - ((math.sin(Theta_rad) * Qreq) / ((math.cos(Theta_rad) * Qreq)+(B**2 * (1 / math.tan(Phi_rad)))))**((3 / 2) * (0.5 + (0.5 * math.cos(2 * Theta_rad))) + ((3 / 2) * 2 * math.sin(Theta_rad)) + 1)
    else:
        F_ig = 1

    # F_tg: Factor de Corrección por Inclinacion en la base-γ
    if TipoMat == "Granular":
        F_tg = (1-(alfa_rad * math.tan(Phi_rad)))**2
    else:
        F_tg = 1

    # F_tq: Factor de Corrección por Inclinacion en la base-q
    if TipoMat == "Granular":
        F_tq = F_tg
    else:
        F_tq = 1

    # F_tc: Factor de Corrección por Inclinacion en la base-c
    if Phi == 0:
        F_tc = 1 - ((2 * alfa_rad)/(math.pi + 2))
    else:
        F_tc = F_tq - (1 - F_tq) / (Nc * math.tan(Phi_rad))

    # F_gq: Factor de Corrección por Inclinacion Terreno-q
    if TipoMat == "Granular":
        F_gq = (1 - math.tan(omega_rad))**2
    else:
        F_gq = 1   

    # F_gc: Factor de Corrección por Inclinacion Terreno-c
    if Phi == 0:
        F_gc = 1-((2 * alfa_rad) / (math.pi + 2))
    else:
        F_gc = F_gq - ((1 - F_gq) / (Nc * math.tan(Phi_rad)))    

    # F_gg: Factor de Corrección por Inclinacion Terreno-γ
    if TipoMat == "Granular":
        F_gg = F_gq
    else:
        F_gg = 1
        
    # Gm_fund_ea: Peso unitario efectivo del suelo de fundación para arrancamiento [Ton/m³]
    if Sumergencia == "Sumergido":
        Gm_fund_ea = 1
    else:
        Gm_fund_ea = max(Gm_fund_ec, Gm_rell_ec)

    # Gm_rell_ea: Peso unitario efectivo del suelo de relleno para arrancamienro [Ton/m³]
    if Sumergencia == "Sumergido":
        Gm_rell_ea = 1
    else:
        Gm_rell_ea = Gm_rell_ec

    # Ka: Carga lateral admisible
    Ka = (math.tan(math.radians(45 - Phi / 2)))**2

    # Kp: Carga lateral admisible
    Kp = (math.tan(math.radians(45 + Phi / 2)))**2

    # Zc: Grieta de Tracción
    Zc = 2 * Cu * math.sqrt(Ka) / (Ka * Gm_fund_ec)

    # C1: Factor de corrección
    if (1 - (0.5 * ((Gm_fund * Df)/(((Qreq + (e * (Gm_conc_e - Gm_fund_ec) * B**2) + ((Df - e) * (Gm_rell_ec - Gm_fund_ec) * B**2))/B**2) - (Gm_fund * Df))))) < 0.5:
        C1 = 0.6 
    else:
        C1 = 1 - (0.5 * ((Gm_fund * Df)/(((Qreq + (e * (Gm_conc_e - Gm_fund_ec) * B**2)+((Df - e) * (Gm_rell_ec - Gm_fund_ec) * B**2))/B**2) - (Gm_fund * Df))))

    # v: Relación de Poisson
    if TipoMat == "Granular":
        v = (1 - math.sin(Phi_rad)) / (1 + (1 - math.sin(Phi_rad)))
    else:
        if Sumergencia == "No Sumergido":
            v = 0.3
        else:
            v = 0.4

    # E: Módulo de Elasticidad (Stround, 1989; Kulhawy 1990) [Ton-f/m²]
    if TipoMat == "Cohesivo":
        if IP < 20:
            E = N * (1075 - 27.5 * IP) * (1 + v)*(1 - 2 * v) / (1 - v) / 9.81
        elif IP < 30:
            E = N * ( 625 -  5.0 * IP) * (1 + v)*(1 - 2 * v) / (1 - v) / 9.81
        elif IP < 40:
            E = N * ( 550 -  2.5 * IP) * (1 + v)*(1 - 2 * v) / (1 - v) / 9.81
        else:
            E = N * ( 450 ) * (1 + v)*(1 - 2 * v) / (1 - v) / 9.81
    else:
        if N <= 15:
            E = 100 * 6 * (N + 6) / 9.81
        else:
            E = (4000 + 100 * 12 * (N - 6)) / 9.81
            
    # F_rq: Factor de Corrección por Rigidez-q
    if TipoMat == "Granular":
        if ((E / (2 * (1 + (0.1 + (0.3 * ((Phi - 25) / 20)))))) * (1 / (Gm_fund_ec * Df * math.tan(Phi_rad))) / (1 + ((E / (2 * (1 + (0.1 + (0.3 * ((Phi - 25) / 20)))))) * (1 / (Gm_fund_ec * Df * math.tan(Phi_rad))) * (0.005 * Gm_fund_ec * Df * (1 - ((Phi - 25) / 20)))))) > (0.5 * math.exp((3.3 - 0.45) * math.atan(math.radians(45 - (Phi / 2))))):
            F_rq = 1
        else:
            F_rq = ((-4.4 * 0.6) * math.tan(Phi_rad)) + ((3.07 * math.sin(Phi_rad)) * (math.log10(2 * ((E / (2 * (1 + (0.1 + (0.3 * ((Phi - 25) / 20)))))) * (1 / (Gm_fund_ec * Df * math.tan(Phi_rad))) / (1 + ((E / (2 * (1 + (0.1 + (0.3 * ((Phi - 25) / 20))))))*(1 / (Gm_fund_ec * Df * math.tan(Phi_rad))) * (0.005 * Gm_fund_ec * Df * (1 - (Phi - 25) / 20)))))))/(1 + math.sin(Phi_rad)))
    else:
        F_rq = 1
        
    # F_rc: Factor de Corrección por Rigidez-c
    if Phi == 0:
        if E / (2 * (1 + v )) > 8.6438:
            F_rc = 1
        else:
            F_rc = 0.32 + 0.12 + (0.6 * math.log10(E / (2 * (1 + v))))
    else:
        F_rc = F_rq - ((1 - F_rq) / (Nc * math.tan(Phi_rad)))   
        
    # F_rg: Factor de Corrección por Rigidez-γ
    F_rg = F_rq   

    # Qadm: Capacidad axial a compresión admisible [Ton-f/m²]
    Qadm = (1 / FS_cp) * ((Nc * Cu * F_sc * F_dc * F_rc * F_ic * F_tc * F_gc) + (Gm_fund_ec * Df * Nq * F_sq * F_dq * F_rq * F_iq * F_tq * F_gq) + (0.5 * Gm_fund_ec * B * Ng * F_sg * F_dg * F_rg * F_ig * F_tg * F_gg))

    # Psi: Ángulo de fricción del cono [Grados]
    if Sumergencia == "Sumergido":
        Psi = 15
    else:
        if Qadm > 10:
            Psi = 30
        else:
            Psi = 20
    Psi_rad = math.radians(Psi)

    # Mest: Momentos de Estabilizacion  [Ton-m]
    Mest = ((Gm_conc * B**2 * e) + ((Gm_rell_ea * ((B**2) * (Df - e))))) * (B / 2)

    # Si: Asentamiento (IEEE Std 691-2001) [mm]
    if TipoMat == "Granular":
        if Phi == 0:
            Si = (0.99 * ((Qreq + (e * (Gm_conc_e - Gm_fund_ec) * B**2) + ((Df - e) * (Gm_rell_ec - Gm_fund_ec) * B**2))/B**2) * B * ((1 - v**2) / E)) * 1000        
        else:
            if ((Qreq + (e * (Gm_conc_e - Gm_fund_ec) * B**2) + ((Df - e) * (Gm_rell_ec - Gm_fund_ec) * B**2)) / B**2) - (Gm_fund * Df) < 0:
                Si = C1 * ((1 + (0.2 * math.log(50 / 0.1)))) * 0.01 *((0.6 / E) * (2 * B)) * 1000
            else:
                Si = C1 * ((1 + (0.2 * math.log(50 / 0.1)))) * (((Qreq + (e * (Gm_conc_e - Gm_fund_ec) * B**2) + ((Df - e) * (Gm_rell_ec - Gm_fund_ec) * B**2)) / B**2) - (Gm_fund * Df)) * ((0.6 / E) * (2 * B)) * 1000    
    elif TipoMat == "Cohesivo" and Sumergencia == "No Sumergido":
        if Phi == 0:
            Si = (0.99 * ((Qreq + (e * (Gm_conc_e - Gm_fund_ec) * B**2) + ((Df - e) * (Gm_rell_ec - Gm_fund_ec) * B**2)) / B**2) * B * ((1 - v**2) / E)) * 1000
        else:
            Si = ((1 - (0.5 * ((Gm_fund * Df) / (((Qreq + (e * (Gm_conc_e - Gm_fund_ec) * B**2)+((Df - e) * (Gm_rell_ec - Gm_fund_ec) * B**2)) / B**2) - (Gm_fund * Df))))) * ((1 + (0.2 * math.log(50 / 0.1)))) * (((Qreq + (e * (Gm_conc_e - Gm_fund_ec) * B**2) + ((Df - e) * (Gm_rell_ec - Gm_fund_ec) * B**2)) / B**2) - (Gm_fund * Df)) * ((0.6 / E) * (2 * B))) * 1000
    elif TipoMat == "Cohesivo" and Sumergencia == "Sumergido":
        if Phi == 0:
            Si = (0.99 * ((Qreq + (e * (Gm_conc_e - Gm_fund_ec) * B**2)+((Df - e) * (Gm_rell_ec - Gm_fund_ec) * B**2)) / B**2) * B * ((1 - v**2) / E)) * 1000 + 21
        else:
            Si = (C1 * ((1 + (0.2 * math.log(50 / 0.1)))) * (((Qreq + (e * (Gm_conc_e - Gm_fund_ec) * B**2) + ((Df - e) * (Gm_rell_ec - Gm_fund_ec) * B**2)) / B**2) - (Gm_fund * Df)) * ((0.6 / E) * (2 * B))) * 1000 + 21

    # Mvolc: Momentos de Vuelco [Ton-m]
    Mvolc = ((0.5 * Ka * Gm_fund_ea * Df**2 * B) * (Df / 3)) + (Freq * (Df + AltPedestal))

    # Faxc_adm: Capacidad axial a compresión admisible [Ton-f]
    Faxc_adm = Qadm * (B**2)

    # Faxt_adm: Capacidad axial a tensión admisible [Ton-f]
    Faxt_adm = ((Gm_conc * B**2 * e) + (Lpe**2 * (Df + AltPedestal) * Gm_conc) + ((Gm_rell_ea * ((B**2 * Df) - (B**2 * e))) + ((1 / 6) * Gm_rell_ea * Df**2 * math.tan(Psi_rad)*((12 * B)+((8 * Df) * math.tan(Psi_rad)))))) / FS_arr

    # Flat_adm: Carga lateral admisible [Ton-f]
    Flat_adm = ((B**2) * Cu * 0.6 + Faxc_adm * math.tan(0.8 * Phi_rad)) / FS_cat + ((e * B * 0.5 * ((Gm_fund_ec * Kp * (2 * Df - e))+(4 * Cu * (Kp)**0.5))))/(FS_cat * 2)
    if Zc < Df:
        Flat_adm -= ((e * B * 0.5 * ((Gm_fund_ec * Ka * (2 * Df - e)) - (4 * Cu * (Ka)**0.5)))) / FS_cat

    # Sp: Asentamiento Permitido [mm]
    if TipoMat == "Cohesivo":
        Sp = 2 * 25.4
    else:
        Sp = 25.4

    # Mest_min: Momento estabilizador Minimo [Ton-m]
    Mest_min = Mest / FS_cat

    return {'N60': N60, 'f1': f1, 'Cu': Cu, 'Gm_fund_ec': Gm_fund_ec, 'Gm_rell_ec': Gm_rell_ec, 'Gm_conc_e': Gm_conc_e,\
            'Phi': Phi, 'Nq': Nq, 'Nc': Nc, 'Ng': Ng, 'F_sc': F_sc, 'F_sq': F_sq, 'F_sg': F_sg, 'F_dq': F_dq, 'F_dc': F_dc,\
            'F_dg': F_dg, 'F_iq': F_iq, 'F_ic': F_ic, 'F_ig': F_ig, 'F_tg': F_tg, 'F_tq': F_tq, 'F_tc': F_tc, 'F_gq': F_gq,\
            'F_gc': F_gc, 'F_gg': F_gg, 'Psi': Psi, 'Gm_fund_ea': Gm_fund_ea, 'Gm_rell_ea': Gm_rell_ea, 'Ka': Ka, 'Kp': Kp,\
            'Zc': Zc, 'C1': C1, 'v': v, 'E': E, 'F_rq': F_rq, 'F_rc': F_rc, 'F_rg': F_rg, 'Qadm': Qadm, 'Mest': Mest,\
            'Si': Si, 'Mvolc': Mvolc, 'Faxc_adm': Faxc_adm, 'Faxt_adm': Faxt_adm, 'Flat_adm': Flat_adm, 'Sp': Sp, 'Mest_min': Mest_min}
